In [120]:
import pandas as pd
import os
import sys
sys.path.append("../")

import seaborn as sns
sns.set_context('poster')
sns.set(rc={'figure.figsize': (16., 9.)})
sns.set_style('whitegrid')

from matplotlib import pyplot as plt
%matplotlib inline
%config Inlinebackend.figure_format = 'retina'

from sklearn import preprocessing
label = preprocessing.LabelEncoder()

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score as cvs
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split as tts
import numpy as np

# Loading the data to train and test the chosen models

In [121]:
test = pd.read_csv("../data/test.csv")
train = pd.read_csv("../data/train.csv")

# Transforming clarity, color and cut columns from strings to identifying numbers

In [122]:
train['clarity'] =  label.fit_transform(train["clarity"])
train['color'] =  label.fit_transform(train["color"])
train['cut'] =  label.fit_transform(train["cut"])

test['clarity'] =  label.fit_transform(test["clarity"])
test['color'] =  label.fit_transform(test["color"])
test['cut'] =  label.fit_transform(test["cut"])

# The variables x, y and z are highly correlated with 'carat' so I decided to drop them in order to train the model

In [123]:
train_noidxyz = train[['carat','cut','color','clarity','depth','table']]
train_noidxyz

,carat,cut,color,clarity,depth,table
0,1.14,2,3,7,61.0,56.0
1,0.76,2,4,5,62.7,57.0
2,0.84,2,3,4,61.4,56.0
3,1.55,2,4,4,62.0,57.0
4,0.30,2,3,3,61.9,57.0
...,...,...,...,...,...,...
40450,0.40,2,1,6,60.8,57.0
40451,0.52,2,6,3,61.6,55.0
40452,1.00,4,1,2,59.4,63.0
40453,0.57,4,1,4,62.9,56.0


# Splitting the training dataset to train and testthe model

In [124]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test  = train_test_split(train_noidxyz ,train.price, test_size=0.2)

# Choosing a model and getting its Root Mean Squared Error

In [134]:
n = [42,45,48] # I used this list to try different variables and check their RMSE
for i in n:
    model= RandomForestRegressor(max_depth= i, min_samples_split= 10, n_estimators= 390)
    model=model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    print(f"RMSE of {i} {np.sqrt(metrics.mean_squared_error(y_test, y_pred))}")

RMSE of 42 547.3841616345304
RMSE of 45 546.547350522786
RMSE of 48 546.6263577375381


# Once the model is trained and tested with a part of the dataset 'train', if the RMSE is good enough I proceed to train it using the whole dataset and choosing the test dataset to get X_test

In [1]:
y_train= train.price
X_train=train_noidxyz
X_test=test[['carat','cut','color','clarity','depth','table']]

NameError: name 'train' is not defined

In [132]:
n=[36,40] # I used this list to try different variables and check their RMSE
for i in n:
    model= RandomForestRegressor(max_depth= i, min_samples_split= 10, n_estimators= 350)
    model=model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    print(f"RMSE of {i} {np.sqrt(metrics.mean_squared_error(y_test, y_pred))}")

RMSE of 36 546.8148751719102
RMSE of 40 547.8938707852477


# I create a new column called 'price' in the test dataset that contains the price predictions from the chosen model

In [105]:
test['price'] = y_pred

# I create the final dataset choosing only the columns that include the price predictions and the id

In [106]:
finpred = test[['id','price']]

In [107]:
finpred.head()

,id,price
0,0,3151.926789
1,1,3094.558637
2,2,3526.925562
3,3,3174.228325
4,4,5290.883317


# I save the prediction in .csv format in the 'predictions' folder

In [109]:
if not os.path.exists("../predictions"):
    os.makedirs("../predictions")
finpred.to_csv("../predictions/predetr1710280.csv", index=False, header=True)